In [2]:
import pandas as pd

prediction = r'D:\retrosynthesis\OpenNMT-py\experiments\results\predictions_on_USPTO50K_test.csv'

df = pd.read_csv(prediction)



In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw
import matplotlib.pyplot as plt


pred_results = dict()
for idx, row in df.iterrows():
    idx = row['index']
    product = row['product']
    synthon = row['synthon']
    target = row['target']
    pred = row['canonical_prediction_1']
    
    if idx not in pred_results:
        pred_results[idx] = {'product': product, 'synthons': []}
    pred_results[idx]['synthons'].append([synthon, target, pred])


pred_results_2th = []
for idx, rxns in pred_results.items():
    flag = True
    pred_results[idx]['okay'] = True
    for rxn in rxns['synthons']:
        if rxn[1] != rxn[2]:
            flag = False
            pred_results[idx]['okay'] = False
    pred_results_2th.append(flag)

print('Second phase Top1 acc = ', sum(pred_results_2th) / len(pred_results_2th))


for idx, rxns in pred_results.items():
    if not pred_results[idx]['okay']:
        continue
    names = ['prod']
    smis = [rxns['product']]
    for i, smi in enumerate(pred_results[idx]['synthons']):
        names.append('synthon_{}'.format(i + 1))
        smis.append(smi[0])
        names.append('target_{}'.format(i + 1))
        smis.append(smi[1])
        names.append('pred_{}'.format(i + 1))
        smis.append(smi[2])
    
    for name, smi in zip(names, smis):
        try:
            mol = Chem.MolFromSmiles(smi, sanitize=False)
            fig = Draw.MolToMPL(mol, kekulize=True, size=(500, 500))
            fig.suptitle(smi, fontsize=20, x=1.25, y=0.1)
            fig.savefig('D:/retrosynthesis/plots/{}_{}_{}.png'.format(idx, name, str(pred_results[idx]['okay'])), bbox_inches='tight')
            plt.cla()
            plt.clf()
            plt.close()
        except:
            pass


Second phase Top1 acc =  0.6560814859197124


In [3]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


error_pred_path = 'D:/retrosynthesis/plots/'
figs = [f for f in os.listdir(error_pred_path) if os.path.isfile(os.path.join(error_pred_path, f))]

names_dict = dict()
for name in figs:
    items = name.split('_')
    if items[0] not in names_dict:
        names_dict[items[0]] = []
    names_dict[items[0]].append(name)
    

for key, items in names_dict.items():
    if len(items) not in [4, 7, 10, 13, 16]:
        continue
    flag = items[0].split('_')[-1][:-4]
    num_reactants = (len(items) - 1) // 3
    
    prod = '{}_prod_{}.png'.format(key, flag)
    plt.figure(figsize=(20, 10))
    plt.subplot(num_reactants + 1, 3, 2)
    plt.imshow(mpimg.imread(os.path.join(error_pred_path, prod)))
    plt.tight_layout()
    plt.axis('off')
    
    for i in range(num_reactants):
        synthon = '{}_synthon_{}_{}.png'.format(key, i + 1, flag)
        plt.subplot(num_reactants + 1, 3, 3 * i + 4)
        plt.imshow(mpimg.imread(os.path.join(error_pred_path, synthon)))
        plt.tight_layout()
        plt.axis('off')
        
        synthon = '{}_target_{}_{}.png'.format(key, i + 1, flag)
        plt.subplot(num_reactants + 1, 3, 3 * i + 4)
        plt.imshow(mpimg.imread(os.path.join(error_pred_path, synthon)))
        plt.tight_layout()
        plt.axis('off')
        
        synthon = '{}_pred_{}_{}.png'.format(key, i + 1, flag)
        plt.subplot(num_reactants + 1, 3, 3 * i + 4)
        plt.imshow(mpimg.imread(os.path.join(error_pred_path, synthon)))
        plt.tight_layout()
        plt.axis('off')
        
    plt.show()
        

# for p, t in zip(pred, tgt):
#     if p.split('_')[0] != t.split('_')[0]:
#         print(p, t)
#         continue
#     plt.figure(figsize=(20, 10))
#     plt.subplot(1, 2, 1)
#     plt.imshow(mpimg.imread(os.path.join(error_pred_path, p)))
#     plt.tight_layout()
#     plt.axis('off')
 
#     plt.subplot(1, 2, 2)
#     plt.imshow(mpimg.imread(os.path.join(error_pred_path, t)))
#     plt.tight_layout()
#     plt.axis('off')
    
#     #plt.show()
#     plt.savefig('D:/error_pred_merge/error_{}'.format(p), bbox_inches='tight')
#     plt.close()


    
